# Spark Setup and Data Load

##Installation of Spark

In [2]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark2.4.5
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
# unzip it
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
# install findspark
!pip install -q findspark
# Google Colab has Java 11 available, test it using below command -
!ls /usr/lib/jvm
#install pyarrow
!pip install -U pyarrow

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark_params = {
"spark.executor.memory" : "4g",
"spark.driver.memory": "4g",
"spark.memory.fraction": "0.9"}
for param, value in spark_params.items():
  spark.conf.set(param, value)

## Load dataset

In [4]:
!wget "https://datasets.imdbws.com/name.basics.tsv.gz"
!wget "https://datasets.imdbws.com/title.akas.tsv.gz"
!wget "https://datasets.imdbws.com/title.basics.tsv.gz"
!wget "https://datasets.imdbws.com/title.crew.tsv.gz"
!wget "https://datasets.imdbws.com/title.episode.tsv.gz"
!wget "https://datasets.imdbws.com/title.principals.tsv.gz"
!wget "https://datasets.imdbws.com/title.ratings.tsv.gz"

--2022-11-22 18:42:58--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.94, 99.84.160.101, 99.84.160.41, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236745635 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz.1’

name.basics.tsv.gz. 100%[===================>] 225.78M  50.5MB/s    in 4.3s    

2022-11-22 18:43:03 (52.2 MB/s) - ‘name.basics.tsv.gz.1’ saved [236745635/236745635]

--2022-11-22 18:43:03--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.94, 99.84.160.101, 99.84.160.41, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288222028 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz.1’

title.akas.tsv.gz.1 100%[===================>] 274.87

In [5]:
title_ratings = spark.read.csv("title.ratings.tsv.gz", sep='\t', header=True)
title_principals = spark.read.csv("title.principals.tsv.gz", sep='\t',header=True)
title_episode = spark.read.csv("title.episode.tsv.gz", sep='\t', header=True)
title_crew = spark.read.csv("title.crew.tsv.gz", sep='\t', header=True)
title_basics = spark.read.csv("title.basics.tsv.gz", sep='\t', header=True)
title_akas = spark.read.csv("title.akas.tsv.gz", sep='\t', header=True)
name_basics = spark.read.csv("name.basics.tsv.gz", sep='\t', header=True)

## Profilowanie danych

Profilowanie danych będziemy przeprowadzać dla próbek ze wszystkich tabel. Wielkość próbki będzie uzależniona od rozmiaru tabeli.

In [6]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
from pandas_profiling import ProfileReport

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


### Tabela title_ratings

In [56]:
sample_title_ratings = title_ratings.sample(.1)

In [57]:
sample_title_ratings = sample_title_ratings .toPandas()
profile_ratings = ProfileReport(sample_title_ratings, title="Pandas Profiling Report - title ratings")
profile_ratings.to_file("title_ratings.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_principals

In [52]:
sample_title_principals = title_principals.sample(.01)

In [53]:
sample_title_principals = sample_title_principals.toPandas()
profile_principals = ProfileReport(sample_title_principals, title="Pandas Profiling Report - title principals")
profile_principals.to_file("title_principals.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_episode

In [16]:
sample_title_episode = title_episode.sample(.1)

In [17]:
sample_title_episode = sample_title_episode.toPandas()
profile_episode = ProfileReport(sample_title_episode, title="Pandas Profiling Report - title episode", minimal=True)
profile_episode.to_file("title_episode.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_crew

In [18]:
sample_title_crew = title_crew.sample(.1)

In [19]:
sample_title_crew = sample_title_crew.toPandas()
profile_crew= ProfileReport(sample_title_crew, title="Pandas Profiling Report - title crew", minimal=True)
profile_crew.to_file("title_crew.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_basics

In [35]:
sample_title_basics= title_basics.sample(.1)

In [36]:
sample_title_basics = sample_title_basics.toPandas()
profile_basics = ProfileReport(sample_title_basics, title="Pandas Profiling Report - title basics", minimal=True)
profile_basics.to_file("title_basics.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_akas

In [60]:
sample_title_akas= title_akas.sample(.05)

In [61]:
sample_title_akas = sample_title_akas.toPandas()
profile_akas = ProfileReport(sample_title_akas, title="Pandas Profiling Report - title akas", minimal=True)
profile_akas.to_file("title_akas.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela name_basics

In [27]:
sample_name_basics= name_basics.sample(.1)

In [28]:
sample_name_basics = sample_name_basics.toPandas()
profile_name = ProfileReport(sample_name_basics, title="Pandas Profiling Report - name basics", minimal=True)
profile_name.to_file("name_basics.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Wnioski



Dla tabeli name_basics możemy zauważyć, że mamy duży brakujących informacji (\N) o roku urodzenia oraz śmierci danej osoby (chociaż to może wynikać z faktu, że dana osoba wciąż żyje). W naszych danych przeważają aktorzy obu płci. Widoczne jest również, że jedna osoba może mieć wiele profesji lub nie mieć ich wcale (21,6% brakujących danych). 

Z tabeli title_akas możemy zauważyć dużą ilość pseudonulli (ponad 84%) dla kolumny types. Podobnie kolumna attributtes jest złożona z prawie samych pseudonulli - innych danych jest tylko 0,7%. Ponadto ponad 82% produkcji pochodzi z 7 regionów, pozostałe niecałe 18% z reszty świata.

Tabela title_basics pokazuje, że znaczącą przewagę w danych mają tvEpisode (ponad 75%), ponieważ mamy rozbicie na kolejne numery epizodów.  Tylko nieco ponad 3% to treści przeznaczone dla dorosłych. Dodatkowo mamy duże braki (\N)  dla endYear - to z kolei wynika z faktu, że dla filmów ten parametr nie jest podawany. Kolejną wybrakowaną kolumną jest runtimeMinutes - ponad 70% wartości \N. W tabeli przeważają dramaty, choć ponownie, dany film może mieć więcej niż jeden gatunek.

Dla tabeli title_crew w kolumnach directors oraz writers ponownie pojawia się problem braków (ponad 40%). 

Tabela title_episode pokazuje, że połowę naszych danych dotyczących seriali stanowią odcinki pochodzące z pierwszych sezonów. Jednak prawie 21% ma brakującą daną dotyczącą numeru sezonu oraz odcinka.

Z tabeli title_principals możemy ponownie wskazać dominację aktorów oraz aktorek, jednak warto zauważyć, że ponad 17% grało siebie. Dla kolumny job mamy ponad 83% pseudonulli. 

Tabela title_ratings pokazuje, że jest ogromna różnica między minimalną a maksymalną liczbą ocen filmu - minumum z próbki to 5, a maksimum 1326662. Nie jest widoczna korelacją pomiędzy liczbą ocen dla filmu a jego oceną.